In [1]:
import datetime
import math
from sympy.interactive import printing
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
printing.init_printing(use_latex=True)

Loading BokehJS ...

# SIE 552 HW #3

This is a space mission design assignment for interplantary travel.  The homework was to create a Type-1 trajectory from Earth to Mars.  Departure from earth is set to be March 22, 2021 with an anticipated arrival on Mars at October 8, 2021.  We're also given the following details about Earth and Mars:

When designing the transfer ellipse, the overall mission has to be considered first.  We know that we want to create an interplanetary travel mission between Earth and Mars.  In order to plan this, we have to know more about those planets:

- Earth at Launch: (March 22, 2021)
    * Longitude = 181.44 deg 
    * Eccentricity = 0.0167
    * SMA =  149,598,020 Km
    * Inclination = 0 deg 
    * Longitude of the Perihelion = 102.958 deg 
    * Longitude of the Ascending Node = 0 deg 
    * Radius = 149,905,909.7 km 
    * Velocity = 29.89 km/sec
    * True Anomaly = 78.48 deg 
    * Flight Path Angle = 0.9348 deg 
    
    
- Mars at Arrival: (October 8, 2021)
    * Longitude = 333.22 deg 
    * Eccentricity = 0.0934
    * SMA =  227,939,133 Km
    * Inclination = 1.849 deg 
    * Longitude of the Perihelion = 336.093 deg 
    * Longitude of the Ascending Node = 49.572 deg 
    * Radius = 206,671,197 km
    * Velocity = 26.94 km/sec
    * True Anomaly = 357.128 deg 
    * Flight Path Angle = - 0.2452 deg
    
We'll store these as `PlanetaryObejct`'s in python to help us later.  

In [2]:
class PlanetaryObject():
    """
    A simple class used to store pertinant information about the plantary object
    """
    def __init__(self, L, e, SMA, i, peri, asc, r, v, anom, fp, mu):
        self.L = L         # Longitude
        self.e = e         # Eccentricity
        self.SMA = SMA     # SMA
        self.i = i         # Inclination
        self.peri = peri   # Longitude of Perihelion
        self.asc = asc     # Longitude of Ascending Node
        self.r = r         # Radius
        self.v = v         # Velocity
        self.anom = anom   # True Anomaly
        self.fp = fp       # Flight Path Angle
        self.mu = mu       # Gravitation parameter

earth = PlanetaryObject(
    181.44,     # Longitude
    0.0167,     # Eccentricity
    149598020,  # SMA
    0,          # Inclination
    102.958,    # Longitude of Perihelion
    0,          # Longitude of Ascending Node
    14905909.7, # Radius
    29.89,      # Velocity
    78.48,      # True Anomaly
    0.9348,     # Flight Path Angle
    398600.4    # Gravitation parameter
)
mars = PlanetaryObject(
    333.22,     # Longitude
    0.0934,     # Eccentricity
    227939133,  # SMA
    1.849,      # Inclination
    336.093,    # Longitude of Perihelion
    49.572,     # Longitude of Ascending Node
    206671197,  # Radius
    26.94,      # Velocity
    357.128,    # True Anomaly
    -0.2452,    # Flight Path Angle
    42828.3     # Gravitation parameter
)

There are also a few fundamental equations we need to know.  These are captured below as python functions.


In [16]:
mu_sun = 132712439935.5
        
def eccentricity(r_1, r_2, theta_1, theta_2):
    """
    Calculates the eccentricity of the transfer ellipse.  This is calculated through 
    the following equation:
    
    .. math::
        \frac {r_2 - r_1} {r_1 * \cos{\theta_1} - r_2 * \cos{\theta_2}}
    
    :param r_1: radius of the departing planetary object
    :param r_2: radius of the arriving planetary object
    :param theta_1: True anomaly of the departing planetary object in degrees
    :param theta_2: True anomaly of the arriving planetary object in degrees
    """
    return (r_2 - r_1) / ((r_1 * math.cos(math.radians(theta_1))) - (r_2 * math.cos(math.radians(theta_2))))

def periapsis_radius(r, e, theta):
    """
    Calculates the periapsis radius of the transfer ellipse.  This is calculated 
    using the following equation:
    
    .. math::
        \frac {r_1 * [1 + e \cos{\theta]}} {1 + e}
    
    :param r: radius of the departing planetary object
    :param e: eccentricity of the transfer ellipse
    """
    return (r * (1 + e * math.cos(math.radians(theta)))) / (1 + e)

def semimajor_axis(r=None, r_a=None, r_p=None, mu=None, V=None, e=None):
    """
    Calculates the semi-major axis of the transfer ellipse.  This is calculated 
    using one of the following equations:
    
    .. math::
        \frac {r_a + r_p} {2}
        
        \frac {\mu r} {2 \mu - V^2 r}
        
        \frac {r_p} {1 - e}
        
        \frac {r_a} {1 + e}
    
    :param r: general radius of the elliptical orbit
    :param r_a: Radius of apoapsis
    :param r_p: Radius of periapsis
    :param mu: gravitation parameter
    :param V: Velocity of the orbiting object
    :param e: Eccentricity of the elliptical orbit
    """
    if r_a != None and r_p != None:
        return (r_a + r_p) / 2
    if mu != None and r !=None and V != None:
        return (mu * r) / (2 * mu - V ** 2 * r)
    if r_p != None and e != None:
        return r_p / (1 - e)
    if r_a != None and e != None:
        return r_a / (1 + e)
    
    # If we reach this point, then the passed in arguments doesn't match
    #    any equations we have defined.  Raise an Error
    raise TypeError("Invalid arguments!")
    

def time_since_periapsis(e, n, theta=None, E=None):
    """
    Calculates the time since the periapsis.  This is calculated using the
    following equation:
    
    .. math::
        \frac {E - e \sin{E}} {n}
        
    If E, isn't defined, it will be calculated using the param theta and 
    the following equation:
    
    ..math:: 
        
        \cos {E} = \frac {e + \cos{\theta}} {1 + e \cos{\theta}}
    
    :param e: eccentricity of the transfer ellipse
    :param n: mean motion
    :param theta: degrees to periapsis
    :param E: eccentric anomaly in radians
    """
    if theta == None and E == None:
        raise TypeError("theta or E MUST be defined")
    if theta != None and E != None:
        raise TypeError("theta OR E must be defined.  Not both")
        
    if E == None:
        cos_E = (e + math.cos(math.radians(theta))) / (1 + e * math.cos(math.radians(theta)))
        E = math.acos(cos_E)
        
    return (E - e * math.sin(E)) / n

def mean_motion(mu, a):
    """
    Calculates the mean motion of an elliptical orbit.  This is calculated 
    using the following equation:
    
    .. math::
        \sqrt{\frac{\mu} {a^3}}
    
    :param mu: gravitation parameter (Mass * Gravitation constant)
    :param a: semimajor axis
    """
    
    return math.sqrt(mu / a ** 3)

def velocity(mu, r, a):
    """
    Calculates the Velocity (V) of an object based on the elliptical orbit.  
    This is calculated using the following equation:
    
    .. math::
        \sqrt{\frac{2 * \mu} {r} - \frac{\mu} {a}}
        
    :param mu: gravitation parameter (Mass * Gravition constant)
    :param a: semimajor axis
    """
    return math.sqrt(2 * mu / r - mu / a)

def flight_path_angle(e, theta):
    """
    Calculates the Flight Path Angle (γ).  This is calculated using
    the following equation:
        
    .. math::
        \tan{γ} = {\frac{e * \sin{\theta}}{1 + 3 * \cos{\theta}}
        
    :param e: eccentricity of the elliptical orbit
    :param theta: 
    """
    tan_y = (e * math.sin(math.radians(theta))) / (1 + e * math.cos(math.radians(theta)))
    return math.atan(tan_y)

def inclination(Omega, L_s, L_t, i):
    a = math.radians(Omega + 180 - L_s)
    b = math.radians(L_t - (180 + Omega))
    alpha = math.radians(180 - i)
    cos_c = math.cos(a) * math.cos(b) + math.sin(a) * math.sin(b) * math.cos(alpha)
    c = math.acos(cos_c)
    sin_i_t = (math.sin(alpha) * math.sin(b)) / math.sin(c)
    return math.asin(sin_i_t)

We'll split this problem up into 3 different sections:

1. Transfer Ellipse
2. Departure Trajectory
3. Arrival Trajectory

This approach is called the "patched conics approach".  

## Designing the Transfer Ellipse

In order to create the transfer ellipse between Earth and Mars, we're going to do the following:

1. Determine the required # of days based off departure and arrival date
2. Calculate the angular difference between Earth and Mars
3. Design the Eccentricity, Periapsis Axis and Semi-Major Axis using an iterative approach
4. Finally, calculating the respective velocities and flight path angles for each planetary object

To find the required number of days to travel to Mars, we simply subtract the launch and arrival dates:

In [4]:
launch_date = datetime.date(2021, 3, 22)
arrival_date = datetime.date(2021, 10, 8)
time_of_flight = arrival_date - launch_date
print(time_of_flight)
time_of_flight = time_of_flight.days  # conversion from datetime to an integer

200 days, 0:00:00


Calculating the angular difference between the two planets is simply a subtraction of their Longitude

In [5]:
delta_L = mars.L - earth.L
print("Mars is {:03.2f}° {} of the Earth position".format(abs(delta_L), 'ahead' if delta_L > 0 else 'behind'))

Mars is 151.78° ahead of the Earth position


Since this difference is < 180°, we know that this is going to be a Type-I trajectory.  Starting with 180° as our trial line of Apsides from Earth, we calculate the eccentricity, periapsis radius and semimajor axis.  From these, we determine the time of flight for both planets and use that to calculate the total time of flight.  We want a time of flight to be near the 200 days we calculated above:

In [6]:
total_TOF = 300 * 3600 * 24
theta = 181  

# storage lists for graphing the results:
thetas = []
tofs = []

while total_TOF / 3600 / 24 > time_of_flight:
    theta -= 1
    e = eccentricity(earth.r, mars.r, earth.anom, mars.anom)
    r = periapsis_radius(earth.r, e, theta)
    a = semimajor_axis(r_p=mars.r, e=e)
    n = mean_motion(mu_sun, a)
    t_E = time_since_periapsis(e, n, theta=theta)
    t_M = time_since_periapsis(e, n, theta=theta+delta_L)
    total_TOF = (t_E + t_M)
    thetas.append(theta)
    tofs.append(total_TOF / 3600 / 24)
    
print("The total time of flight for Earth θ = {:.2f}° and Mars θ = {:.2f}° is {:.2f} days".format(
    theta, theta+delta_L, total_TOF/3600/24
))

The total time of flight for Earth θ = 21.00° and Mars θ = 172.78° is 199.65 days


In [7]:
p = figure(title='Time of Flight Based On Selected Line of Apsides', width=600, height=600)
p.hbar(y=thetas, height=0.5, left=180, right=tofs, color="firebrick")
p.xaxis.axis_label = 'Time of Flight (Days)'
p.yaxis.axis_label = 'Trial line of Apsides (θ°)'

show(p)

Now that we've iteratively found a transfer ellipse with a travel time near the required 200 days, we examine the found eccentricity, periapsis radius and semimajor axis:

In [13]:
print("Eccentricity:     {:05.2f}".format(e))
print("Periapsis Radius: {:05.2f} x 10^6 km".format(r/10e6))
print("Semi-Major Axis:  {:05.2f} x 10^6 km".format(a/10e6))

Eccentricity:     -0.94
Periapsis Radius: 03.12 x 10^6 km
Semi-Major Axis:  10.64 x 10^6 km


Next we determine the Velocities near Earth and Mars as well as their respective flight path angles (γ):

In [9]:
V_ne = velocity(mu_sun, earth.r, a)
V_nm = velocity(mu_sun, mars.r, a)
y_ne = flight_path_angle(e, theta)
y_nm = flight_path_angle(e, theta + delta_L)
print("Velocity near Earth: {:.2f} km/s; \tVelocity near Mars: {:.2f} km/s\nγ_e: {:.2f};\t\t\t\tγ_m: {:.2f}".format(
        V_ne, V_nm, y_ne, y_nm
))

Velocity near Earth: 128.68 km/s; 	Velocity near Mars: 6.07 km/s
γ_e: -1.23;				γ_m: -0.06


## The Departure Trajectory

The departure trajectory entails the following:

* Determining the plane change
* Calculating $V_{HE}$ and $C3$

$V_{HE}$ is the vector required for the spacecraft to leave the Earth's gravitational pull, and enter into the calculated transfer ellipse above.

We'll use the function defined above to find the inclination of the transfer plane $i_t$:

In [17]:
i_t = inclination(mars.asc, earth.L, mars.L, mars.i)
print("i_t = {:.2f}°".format(math.degrees(i_t)))

i_t = 3.80°


Using $i_t$, we can now determine the $V_{HE}$ and $C3$ of the departing trajectory:

In [18]:
cos_alpha_2 = math.cos(i_t) * math.cos(earth.fp + abs(y_ne))
alpha_2 = math.acos(cos_alpha_2)
C3 = earth.v ** 2 + V_ne ** 2 - 2 * earth.v * V_ne * math.cos(alpha_2)
V_he = math.sqrt(C3)
print("C3 = {:.2f} km^2/s^2; V_he = {:.2f} km/s".format(C3, V_he))

C3 = 21745.76 km^2/s^2; V_he = 147.46 km/s


## The Arrival Trajectory

Similar to what was done for the departure trajectory, we'll calculate the plane change and then determine $V_\infty$.  